# `aiterutils` tutorial

A functional programming toolkit for manipulation of asynchronous iterators in python >3.5

It has two types of operations:

1. Iterator functions
   * `au.map(fn, aiter): aiter`
   * `au.each(fn, aiter): coroutine`
   * `au.filter(fn, aiter): aiter`
   * `au.merge([aiter...]): aiter`
   * `au.bifurcate(fn, aiter): (aiter, aiter)`
   * `au.branch([fn...], aiter): (aiter...)`

2. Dynamic streams
   * `au.Stream(aiter)'

Dynamics streams can have their undelying object methods invocated implicitly

```
capitals = au.Stream(['a','b','c']).upper()
```

In [3]:
# if executing this notebook, downgrade tornado 
# as tornado does not play nice with asyncio 
# ! pip install tornado==4.5.3 

## 1. Setup

In [12]:
import functools as ft
import asyncio

import aiterutils as au

async def symbol_stream(symbols, interval):
    for symbol in symbols:
        yield symbol
        await asyncio.sleep(interval)
    
smiley = lambda: symbol_stream(["🙂"]*5, 1)
sunny  = lambda: symbol_stream(["☀️"]*5, 0.5)
clown  = lambda: symbol_stream(["🤡"]*5, 0.2)
team   = lambda: au.merge([smiley(), sunny(), clown()])

single_line_print = ft.partial(print, end='')

loop   = asyncio.get_event_loop()

## 2. Iterator functions

### `au.map(fn, aiter): aiter`

In [3]:
stream = au.map(lambda s: s + '!', team())
app    = au.println(stream, end='')

loop.run_until_complete(app)

🙂!☀️!🤡!🤡!🤡!☀️!🤡!🤡!🙂!☀️!☀️!🙂!☀️!🙂!🙂!

### `au.each(fn, aiter): coroutine`

In [4]:
import functools as ft

app = au.each(single_line_print, team())
loop.run_until_complete(app)

🙂☀️🤡🤡🤡☀️🤡🤡🙂☀️☀️🙂☀️🙂🙂

### `au.filter(fn, aiter): aiter`

In [5]:
stream = au.filter(lambda s: s == '☀️', team())
app    = au.println(stream, end="")
loop.run_until_complete(app)

☀️☀️☀️☀️☀️

### `au.merge([aiter...]): aiter`

In [6]:
stream = au.merge([smiley(), sunny()])
app    = au.println(stream, end="")
loop.run_until_complete(app)

🙂☀️☀️🙂☀️☀️🙂☀️🙂🙂

### `au.bifurcate(fn, aiter): (aiter, aiter)`

In [7]:
smile_stream, other_stream = au.bifurcate(lambda s: s == '🙂', team())

loop.run_until_complete(au.println(smile_stream, end=""))
loop.run_until_complete(au.println(other_stream, end=""))

🙂🙂🙂🙂🙂☀️🤡🤡🤡☀️🤡🤡☀️☀️☀️

### `au.branch([fn...], aiter): (aiter...)`

In [8]:
filters = [
    lambda s: s == '🙂',
    lambda s: s == '☀️',
    lambda s: s == '🤡'
]

smile_stream, sun_stream, clown_stream = au.branch(filters, team())

loop.run_until_complete(au.println(smile_stream, end=''))
loop.run_until_complete(au.println(sun_stream, end=''))
loop.run_until_complete(au.println(clown_stream, end=''))

🙂🙂🙂🙂🙂☀️☀️☀️☀️☀️🤡🤡🤡🤡🤡

## 3. Dynamic Streams

### 3.1 map and each methods

In [9]:
#dynamic streams have regular map an each methods

app = (au.Stream(team())
         .map(lambda s: s+'!')
         .each(single_line_print))

loop.run_until_complete(app)

🙂!☀️!🤡!🤡!🤡!☀️!🤡!🤡!🙂!☀️!☀️!🙂!☀️!🙂!🙂!

### 3.2 operator overloading 

In [10]:
# dynamics streams can have underlying object methods called implicitly

stream = au.Stream(symbol_stream([{"key-2" : 1, "key-2" : 2}]*10, 0.5))
value2 = stream['key-2'] + 3 / 5

app = au.println(value2, end='')
loop.run_until_complete(app)


2.62.62.62.62.62.62.62.62.62.6

### 3.3 dynamic method invocation

In [16]:
stream = au.Stream(symbol_stream([{"key-1" : 1, "key-2" : 2}]*5, 0.5))

keys = stream.keys()
key = keys.map(lambda key: list(key))
key = key[-1]
key = key.upper().encode('utf-8')

app = au.println(key, end='')

loop.run_until_complete(app)

b'KEY-2'b'KEY-2'b'KEY-2'b'KEY-2'b'KEY-2'